<a href="https://colab.research.google.com/github/Progc-unlam/material-progc/blob/main/Ejemplos/Comunicacion/Python/Clase_07_05_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import signal
import sys
import time

CHILD = 0

def manejador(sig, frame):
    sys.stdout.write(f"Recibido {signal.Signals(sig).name}\n")
    #Info básica del objeto frame (Frame Object o PyFrameObject)
    #print(frame)

def main():
    signal.signal(signal.SIGCONT, manejador)

    for i in range(2):
        process = os.fork()
        if process < 0:
            print("Error al crear el proceso ",i+1)
            sys.exit(1)

        if process == CHILD:
           signal.raise_signal(signal.SIGSTOP)
           os._exit(0)

    time.sleep(2)
    os.killpg(os.getpgrp(), signal.SIGCONT)

    for i in range(2):
        os.wait()

if __name__ == '__main__':
    main()

Recibido SIGCONT
Recibido SIGCONT
<frame at 0x7c57ceb36c20, file '<ipython-input-32-54e67b152623>', line 23, code main><frame at 0x7c57ceaa9be0, file '<ipython-input-32-54e67b152623>', line 23, code main>

Recibido SIGCONT
<frame at 0x7c57ceb36c20, file '<ipython-input-32-54e67b152623>', line 27, code main>


In [ ]:
from multiprocessing import Pipe
import os
import sys

CHILD = 0

def main():
    recepcion, envio = Pipe(False)

    process = os.fork()
    if process < 0:
        sys.exit("Error al crear el nuevo proceso")

    if process == CHILD:
        envio.close()
        print("Recibido: ", recepcion.recv())
        recepcion.close()
        os._exit(0)
    else:
        recepcion.close()
        envio.send((1, 'Yerba "La Tranquera"'))
        envio.close()
        os.wait()

if __name__ == '__main__':
    main()

Recibido:  (1, 'Yerba "La Tranquera"')


In [ ]:
import pickle

data = {"nombre": "Juan", "edad": 30}

serialized_data = pickle.dumps(data)
print(f"Objeto serializado: {serialized_data}")

deserialized_data = pickle.loads(serialized_data)
print(f"Objeto deserializado: {deserialized_data}")

Objeto serializado: b'\x80\x04\x95\x1e\x00\x00\x00\x00\x00\x00\x00}\x94(\x8c\x06nombre\x94\x8c\x04Juan\x94\x8c\x04edad\x94K\x1eu.'
Objeto deserializado: {'nombre': 'Juan', 'edad': 30}


In [ ]:
import pickle

class Persona:
    def __init__(self, nombre, edad):
        self.nombre = nombre
        self.edad = edad

# Lista de objetos
personas = [
    Persona("Ana", 30),
    Persona("Luis", 25),
    Persona("Carla", 40)
]

# Guardar la lista completa en un archivo
with open("personas.pkl", "wb") as f:
    pickle.dump(personas, f)

# Leer la lista completa
with open("personas.pkl", "rb") as f:
    personas_recuperadas = pickle.load(f)

for persona in personas_recuperadas:
    print(persona.nombre, persona.edad)

Ana 30
Luis 25
Carla 40


In [ ]:
%%writefile A.py
import sys
import os

try:
    os.mkfifo("./fifo", 0o600)  # Crear el FIFO si no existe
except OSError as err:
    print(f"Error al crear fifo: {err}")
    sys.exit(1)

try:
    fifo = os.open("./fifo", os.O_WRONLY)  # Abrir el FIFO para escribir
    os.write(fifo, "Café La Morenita".encode())  # Escribir en el FIFO
    os.close(fifo)  # Cerrar el FIFO
    os.unlink("./fifo")  # Eliminar el FIFO
except OSError as err:
    print(f"Error al trabajar con el FIFO: {err}")
    sys.exit(1)

Overwriting A.py


In [ ]:
%%writefile B.py
import os

BYTES = 1024

# Intentar abrir el FIFO para lectura
try:
    fifo = os.open("./fifo", os.O_RDONLY)

    # Leer datos del FIFO (hasta BYTES bytes)
    data = os.read(fifo, BYTES)

    # Imprimir los datos leídos después de decodificar
    print(data.decode())

    # Cerrar el FIFO después de leer
    os.close(fifo)

except OSError as e:
    print(f"Error al abrir o leer el FIFO: {e}")

Overwriting B.py


In [ ]:
!nohup python A.py &

nohup: appending output to 'nohup.out'


In [ ]:
!python B.py

Café La Morenita


In [ ]:
%%writefile SHMA.py
from multiprocessing import shared_memory
import time
import numpy

ENTEROS = 5
TAM_ENT = numpy.dtype(numpy.int32).itemsize  # Calcular bytes por entero

try:
    mc = shared_memory.SharedMemory(name="mi_mem", create=True, size=TAM_ENT*ENTEROS)
    print(f"Memoria compartida creada con nombre: {mc.name}")
except FileExistsError:
    print("La memoria compartida ya existe.")

for i in range(ENTEROS):
    mc.buf[i] = i+1

time.sleep(5)

mc.close()
mc.unlink()
print("Finalizando SHMA.py")

Overwriting SHMA.py


In [ ]:
%%writefile SHMB.py
from multiprocessing import shared_memory
from multiprocessing.resource_tracker import unregister

ENTEROS = 5

try:
    mc = shared_memory.SharedMemory(name="mi_mem")
    print(f"Memoria compartida accedida: {mc.name}")
except FileNotFoundError:
    print("La memoria compartida no existe.")
    sys.exit(1)

for i in range(ENTEROS):
    print( mc.buf[i] )

mc.close()
#Para evitar el comportamiento reportado en:
#https://bugs.python.org/issue39959#msg368770
unregister(mc._name, "shared_memory")
print("Finalizando SHMB.py")

Overwriting SHMB.py


In [ ]:
!nohup python SHMA.py &

nohup: appending output to 'nohup.out'


In [ ]:
!python SHMB.py

Memoria compartida accedida: mi_mem
1
2
3
4
5
Finalizando SHMB.py
/usr/lib/python3.11/multiprocessing/resource_tracker.py:254: UserWarning: resource_tracker: There appear to be 1 leaked shared_memory objects to clean up at shutdown
  warnings.warn('resource_tracker: There appear to be %d '


In [ ]:
%%writefile SERVER.py
import socket

MAX_PEND = 10
PUERTO = 5000

def servidor():
    ss = socket.socket()

    try:
        ss.bind(("127.0.0.1", PUERTO))
    except OSError as e:
        print(f"Error al enlazar el socket: {e}")
        return

    ss.listen(MAX_PEND)
    print(f"Servidor escuchando en 127.0.0.1:{PUERTO}...")

    cs, dir = ss.accept()
    print(f"Conexión aceptada desde {dir}")

    try:
        while True:
            data = cs.recv(1024)
            if not data:
                break
            recibi = data.decode("utf-8")
            if recibi.strip() == "fin":
                break
            print("Recibido:", recibi)
    except Exception as e:
        print(f"Error durante la comunicación: {e}")
    finally:
        cs.close()
        ss.close()
        print("Conexión cerrada.")

if __name__ == '__main__':
    servidor()

Overwriting SERVER.py


In [ ]:
%%writefile CLIENT.py
import socket

PUERTO = 5000

def cliente():
    cs = socket.socket()

    try:
        cs.connect(("127.0.0.1", PUERTO))
    except ConnectionRefusedError:
        print("Error: no se pudo conectar al servidor.")
        return

    print("Conectado al servidor. Escribe 'fin' para salir.")
    while True:
        envie = input("> ").strip()
        cs.send(envie.encode("utf-8"))
        if envie == "fin":
            break

    print("Conexión cerrada.")
    cs.close()

if __name__ == '__main__':
    cliente()

Overwriting CLIENT.py


In [ ]:
!nohup python SERVER.py &

nohup: appending output to 'nohup.out'


In [ ]:
!python CLIENT.py

Conectado al servidor. Escribe 'fin' para salir.
> pepe
> pipi
> popo
> fin
Conexión cerrada.


In [ ]:
%%writefile SERVER_IMG.py
import socket

PUERTO = 4000
IMAGEN_RECV_PATH = "chaos_world_copia.png"  # Ruta donde se guardará la imagen recibida

def servidor():
    ss = socket.socket()
    ss.bind(("127.0.0.1", PUERTO))
    ss.listen(1)

    print("Esperando conexión...")
    cs, dir = ss.accept()
    print(f"Conexión aceptada desde {dir}")

    with open(IMAGEN_RECV_PATH, "wb") as file:
        while True:
            # Recibir los fragmentos de 1024 bytes
            print("recibido")
            fragmento = cs.recv(1024)
            if not fragmento:
                break  # Si no hay más datos, terminamos
            file.write(fragmento)  # Escribir el fragmento en el archivo

    print("Imagen recibida y guardada.")
    cs.close()
    ss.close()

if __name__ == '__main__':
    servidor()

Overwriting SERVER_IMG.py


In [ ]:
%%writefile CLIENT_IMG.py
import socket

PUERTO = 4000
IMAGEN_PATH = "chaos_world.png"  # Ruta de la imagen que quieres enviar

def cliente():
    cs = socket.socket()
    cs.connect(("127.0.0.1", PUERTO))

    # Abrir el archivo de la imagen en modo binario
    with open(IMAGEN_PATH, "rb") as file:
        while True:
            # Leer un fragmento de 1024 bytes
            fragmento = file.read(1024)
            if not fragmento:
                break  # Si ya no hay más datos, terminamos
            cs.send(fragmento)  # Enviar el fragmento

    print("Imagen enviada.")
    cs.close()

if __name__ == '__main__':
    cliente()

Overwriting CLIENT_IMG.py


In [ ]:
!nohup python SERVER_IMG.py &

nohup: appending output to 'nohup.out'


In [ ]:
!python CLIENT_IMG.py

Imagen enviada.
